In [3]:
#   LetterGrade5.py + deepSeek (https://chat.deepseek.com/a/chat/s/e12686da-ddda-4810-b5e6-9f4c09141a07)
#   + my debug for edge '3.475' by adding 3.5, 2.5, 1.5 to GRADE_POINTS for 0.25 interval

import warnings

import sys

def custom_excepthook(type, value, traceback):
    if type is AssertionError:
        print(f"Assertion failed: {value}")
    sys.__excepthook__(type, value, traceback)  # Default handler

sys.excepthook = custom_excepthook

class LetterGrade:  #  https://x.com/i/grok?conversation=1881109924950020326
    # Mapping of letter grades to numeric points
    GRADE_POINTS = {
        'A+': 4.25, 'A': 4.0, 'A-': 3.75,  # 'A-/B+': 3.5,  ###  uncomment for fix
        'B+': 3.25, 'B': 3.0, 'B-': 2.75,  # 'B-/C+': 2.5,  ###  uncomment for fix
        'C+': 2.25, 'C': 2.0, 'C-': 1.75,  # 'C-/D+': 1.5,  ###  uncomment for fix
        'D+': 1.25, 'D': 1.0, 'D-': 0.75,
        'F': 0.0
    }
    POINTS_TO_GRADE = {v: k for k, v in GRADE_POINTS.items()}
    GRADE_ORDER = ['F', 'D-', 'D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+']

    def __init__(self, grade):
        self.original_grade = grade
        self.numeric_value = self._to_numeric(grade)

    def _to_numeric(self, grade):
        if isinstance(grade, LetterGrade):  # New check for LetterGrade objects (per DeepSeek)
            return grade.numeric_value
        if isinstance(grade, (int, float)):
            if not 0 <= grade <= 4.25:
                # raise ValueError(f"Numeric value {grade} out of valid range (0 to 4.25)")
                warnings.warn(
                    f"Numeric value {grade} is outside valid range (0–4.25)",
                    RuntimeWarning
                )
            return float(grade)
        if not isinstance(grade, str):
            raise TypeError(f"Unsupported grade type: {type(grade)}")
        grade = grade.strip()
        if '/' in grade:
            grade1, grade2 = [g.strip() for g in grade.split('/')]
            if grade1 not in self.GRADE_POINTS or grade2 not in self.GRADE_POINTS:
                raise ValueError(f"Invalid composite grade: {grade}")
            return (self.GRADE_POINTS[grade1] + self.GRADE_POINTS[grade2]) / 2
        if grade not in self.GRADE_POINTS:
            try:
                value = float(grade)
                if not 0 <= value <= 4.25:
                    raise ValueError(f"Numeric value {value} out of valid range (0 to 4.25)")
                return value
            except ValueError:
                raise ValueError(f"Invalid grade: {grade}")
        return self.GRADE_POINTS[grade]

    def to_letter(self):
        '''  #  original LG5.py
        """Convert numeric value to closest valid letter or adjacent composite grade, higher grade first."""
        if self.numeric_value in self.POINTS_TO_GRADE:
            return self.POINTS_TO_GRADE[self.numeric_value]
        # Find all exact composite matches, prioritize adjacent ones
        composites = []
        for i, g1 in enumerate(self.GRADE_ORDER):
            for g2 in self.GRADE_ORDER[i:]:
                avg = (self.GRADE_POINTS[g1] + self.GRADE_POINTS[g2]) / 2
                if abs(avg - self.numeric_value) < 1e-10:  # Exact match
                    distance = abs(self.GRADE_ORDER.index(g1) - self.GRADE_ORDER.index(g2))
                    # Order by higher grade first
                    if self.GRADE_POINTS[g1] >= self.GRADE_POINTS[g2]:
                        composite = f"{g1}/{g2}"
                    else:
                        composite = f"{g2}/{g1}"
                    composites.append((distance, composite))
        if composites:
            composites.sort()  # Smallest distance first
            return composites[0][1]
        # Fallback to closest single grade
        closest_value = min(self.POINTS_TO_GRADE.keys(), key=lambda x: abs(x - self.numeric_value))
        return self.POINTS_TO_GRADE[closest_value]
        '''

        ''' # deepseek itr1:
        """Convert numeric value to closest valid letter or adjacent composite grade, higher grade first."""
        if self.numeric_value in self.POINTS_TO_GRADE:
            return self.POINTS_TO_GRADE[self.numeric_value]
        # Find all possible composite grade pairs that average to this value
        composites = []  #  based on DeepSeek
        for i, g1 in enumerate(self.GRADE_ORDER):
            for j, g2 in enumerate(self.GRADE_ORDER[i+1:], start=i+1):
                avg = (self.GRADE_POINTS[g1] + self.GRADE_POINTS[g2]) / 2
                if abs(avg - self.numeric_value) < 1e-10:  # Exact match
                    # Sort grades in descending order (higher grade first)
                    if self.GRADE_POINTS[g1] > self.GRADE_POINTS[g2]:
                        composite = f"{g1}/{g2}"
                    else:
                        composite = f"{g2}/{g1}"
                    composites.append((abs(i-j), composite))  # Store distance and composite
        if composites:
            # Pick the pair with smallest distance between grades
            composites.sort()
            return composites[0][1]
        # Fallback to closest single grade
        closest_value = min(self.POINTS_TO_GRADE.keys(), key=lambda x: abs(x - self.numeric_value))
        return self.POINTS_TO_GRADE[closest_value]
        '''
        if self.numeric_value in self.POINTS_TO_GRADE:
            return self.POINTS_TO_GRADE[self.numeric_value]
    
        # First check for exact midpoints between adjacent grades
        for i in range(len(self.GRADE_ORDER)-1):
            g1 = self.GRADE_ORDER[i]
            g2 = self.GRADE_ORDER[i+1]
            midpoint = (self.GRADE_POINTS[g1] + self.GRADE_POINTS[g2]) / 2
            if abs(midpoint - self.numeric_value) < 1e-10:  # Exact match
#                print( "DBG:" , self.numeric_value , midpoint , end = '\t' )
                # Return higher grade first
                return f"{g1}/{g2}" if self.GRADE_POINTS[g1] > self.GRADE_POINTS[g2] else f"{g2}/{g1}"
    
        # Then check for other possible composite grades
        composites = []
        for i, g1 in enumerate(self.GRADE_ORDER):
            for j, g2 in enumerate(self.GRADE_ORDER[i+1:], start=i+1):
                avg = (self.GRADE_POINTS[g1] + self.GRADE_POINTS[g2]) / 2
                if abs(avg - self.numeric_value) < 1e-10: # Exact match vs 0.0625 closest match
                    # Store distance and composite (higher grade first)
#                    print( "DBG2:" , self.numeric_value , midpoint , end = '\t' )
                    composite = f"{g1}/{g2}" if self.GRADE_POINTS[g1] > self.GRADE_POINTS[g2] else f"{g2}/{g1}"
                    composites.append((abs(i-j), composite))
        if composites:
            composites.sort()  # Prefer closest pairs
            return composites[0][1]
    
        # Fallback to closest single grade
        closest_value = min(self.POINTS_TO_GRADE.keys(), key=lambda x: abs(x - self.numeric_value))
        return self.POINTS_TO_GRADE[closest_value]
        '''
        """Convert numeric value to closest valid letter or adjacent composite grade, higher grade first."""
        # First check exact matches (including midpoints)
        if self.numeric_value in self.POINTS_TO_GRADE:
            return self.POINTS_TO_GRADE[self.numeric_value]
        
        print( self.numeric_value , abs( 0.5 - abs( self.numeric_value - int( self.numeric_value ) ) ) )
        if ( abs( 0.0625 - abs( 0.5 - abs( self.numeric_value - int( self.numeric_value ) ) ) < 1e-10 ) :
            self.numeric_value += 0.01
    
        # Check if value is exactly between two adjacent grades
        for i in range(len(self.GRADE_ORDER)-1):
            lower = self.GRADE_POINTS[self.GRADE_ORDER[i]]
            upper = self.GRADE_POINTS[self.GRADE_ORDER[i+1]]
            midpoint = (lower + upper) / 2
            if abs(self.numeric_value - midpoint) < 1e-10:
                # Return higher grade first
                return f"{self.GRADE_ORDER[i+1]}/{self.GRADE_ORDER[i]}" if lower < upper else f"{self.GRADE_ORDER[i]}/{self.GRADE_ORDER[i+1]}"
    
        # Fallback to nearest single grade
        closest_value = min(self.POINTS_TO_GRADE.keys(), key=lambda x: abs(x - self.numeric_value))
        return self.POINTS_TO_GRADE[closest_value]
        '''
        
    def __float__(self):
        return float(self.numeric_value)

    def __add__(self, other):
        '''
        if isinstance(other, (int, float)):
            new_value = self.numeric_value + other
            if 0 <= new_value <= 4.25:
                return LetterGrade.from_numeric(new_value)
            return new_value
        elif isinstance(other, LetterGrade):
            new_value = self.numeric_value + other.numeric_value
            if 0 <= new_value <= 4.25:
                return LetterGrade.from_numeric(new_value)
            return new_value
        raise TypeError(f"Cannot add {type(other)} to LetterGrade")
        '''
        if isinstance(other, (int, float)):
            new_value = self.numeric_value + other
        elif isinstance(other, LetterGrade):
            new_value = self.numeric_value + other.numeric_value
        else:
            raise TypeError(f"Cannot add {type(other)} to LetterGrade")
    
        # Return raw numeric value if outside valid range
        if not (0 <= new_value <= 4.25):
            return new_value
        return LetterGrade.from_numeric(new_value)

    def __sub__(self, other):
        if isinstance(other, (int, float)):
            new_value = self.numeric_value - other
            if 0 <= new_value <= 4.25:
                return LetterGrade.from_numeric(new_value)
            return new_value
        elif isinstance(other, LetterGrade):
            return self.numeric_value - other.numeric_value
        raise TypeError(f"Cannot subtract {type(other)} from LetterGrade")
        
    def __mul__(self, other):  #  https://chat.deepseek.com/a/chat/s/e12686da-ddda-4810-b5e6-9f4c09141a07
        '''
        if isinstance(other, (int, float)):
            new_value = self.numeric_value * other
            if 0 <= new_value <= 4.25:
                return LetterGrade.from_numeric(new_value)
            return new_value
        raise TypeError(f"Cannot multiply LetterGrade by {type(other)}")
        '''
        if not isinstance(other, (int, float)):
            raise TypeError(f"Cannot multiply by {type(scalar)}")
        new_value = self.numeric_value * other
        if 0 <= new_value <= 4.25:
            return LetterGrade.from_numeric(new_value)
        return new_value  # Return raw number if out of bounds

    def __truediv__(self, other):
        if isinstance(other, (int, float)):
            if other == 0:
                raise ZeroDivisionError("Cannot divide by zero")
            new_value = self.numeric_value / other
            if 0 <= new_value <= 4.25:
                return LetterGrade.from_numeric(new_value)
            return new_value
        raise TypeError(f"Cannot divide LetterGrade by {type(other)}")

    def __rmul__(self, other):
        return self.__mul__(other)

    def __rtruediv__(self, other):
        if isinstance(other, (int, float)):
            if self.numeric_value == 0:
                raise ZeroDivisionError("Cannot divide by zero")
            new_value = other / self.numeric_value
            if 0 <= new_value <= 4.25:
                return LetterGrade.from_numeric(new_value)
            return new_value
        raise TypeError(f"Cannot divide {type(other)} by LetterGrade")

    @classmethod
    def median(cls, grades):
        if not grades:
            raise ValueError("Grade list cannot be empty")
        numeric_grades = [cls(g).numeric_value for g in grades]
        numeric_grades.sort()
        n = len(numeric_grades)
        mid = n // 2
        if n % 2 == 0:
            median_value = (numeric_grades[mid - 1] + numeric_grades[mid]) / 2
            return cls.from_numeric(median_value).to_letter()
        else:
            return cls.from_numeric(numeric_grades[mid]).to_letter()

    @classmethod
    def five_number_summary(cls, grades, q13='ee'):
        if not grades:
            raise ValueError("Grade list cannot be empty")
        if q13 not in ['ii', 'ie', 'ee']:
            raise ValueError("q13 must be 'ii', 'ie', or 'ee'")
        
        all_numeric = all(isinstance(g, (int, float)) for g in grades)
        in_range = all_numeric and all(0 <= g <= 4.25 for g in grades)
        
        if all_numeric and not in_range:
            numeric_grades = sorted(float(g) for g in grades)
            n = len(numeric_grades)
            mid = n // 2
            if n % 2 == 0:
                median_val = (numeric_grades[mid - 1] + numeric_grades[mid]) / 2
            else:
                median_val = numeric_grades[mid]
            
            if q13 == 'ii':
                lower_half = numeric_grades[:mid + 1]
                upper_half = numeric_grades[mid:]
            elif q13 == 'ie':
                lower_half = numeric_grades[:mid + (n % 2)]
                upper_half = numeric_grades[mid + (n % 2):]
            elif q13 == 'ee':
                lower_half = numeric_grades[:mid]
                upper_half = numeric_grades[mid + 1:]
            
            lower_mid = len(lower_half) // 2
            upper_mid = len(upper_half) // 2
            q1_val = (lower_half[lower_mid - 1] + lower_half[lower_mid]) / 2 if len(lower_half) % 2 == 0 and len(lower_half) > 0 else lower_half[lower_mid] if len(lower_half) > 0 else numeric_grades[0]
            q3_val = (upper_half[upper_mid - 1] + upper_half[upper_mid]) / 2 if len(upper_half) % 2 == 0 and len(upper_half) > 0 else upper_half[upper_mid] if len(upper_half) > 0 else numeric_grades[-1]
            
            return (numeric_grades[0], q1_val, median_val, q3_val, numeric_grades[-1])
        
        numeric_grades = [cls(g).numeric_value for g in grades]
        numeric_grades.sort()
        n = len(numeric_grades)
        
        min_val = numeric_grades[0]
        max_val = numeric_grades[-1]
        
        mid = n // 2
        if n % 2 == 0:
            median_val = (numeric_grades[mid - 1] + numeric_grades[mid]) / 2
        else:
            median_val = numeric_grades[mid]
        
        if q13 == 'ii':
            lower_half = numeric_grades[:mid + 1]
            upper_half = numeric_grades[mid:]
        elif q13 == 'ie':
            lower_half = numeric_grades[:mid + (n % 2)]
            upper_half = numeric_grades[mid + (n % 2):]
        elif q13 == 'ee':
            lower_half = numeric_grades[:mid]
            upper_half = numeric_grades[mid + 1:]
        
        lower_mid = len(lower_half) // 2
        upper_mid = len(upper_half) // 2
        q1_val = (lower_half[lower_mid - 1] + lower_half[lower_mid]) / 2 if len(lower_half) % 2 == 0 and len(lower_half) > 0 else lower_half[lower_mid] if len(lower_half) > 0 else min_val
        q3_val = (upper_half[upper_mid - 1] + upper_half[upper_mid]) / 2 if len(upper_half) % 2 == 0 and len(upper_half) > 0 else upper_half[upper_mid] if len(upper_half) > 0 else max_val

        return (
            cls.from_numeric(min_val).to_letter(),
            cls.from_numeric(q1_val).to_letter(),
            cls.from_numeric(median_val).to_letter(),
            cls.from_numeric(q3_val).to_letter(),
            cls.from_numeric(max_val).to_letter()
        )

    @classmethod
    def dot_product(cls, weights, grades):
        if not weights or not grades:
            raise ValueError("Weights and grades lists cannot be empty")
        if len(weights) != len(grades):
            raise ValueError(f"Length mismatch: weights ({len(weights)}) and grades ({len(grades)}) must be equal")
        if not all(isinstance(w, (int, float)) for w in weights):
            raise TypeError("All weights must be numeric")
        
        numeric_grades = [cls(g).numeric_value for g in grades]
        result = sum(w * g for w, g in zip(weights, numeric_grades))
        
        if 0 <= result <= 4.25:
            return cls.from_numeric(result)
        return result

    @classmethod
    def to_letter_grades(cls, digits):
        if not digits:
            raise ValueError("Digit list cannot be empty")
        # return [cls.from_numeric(d).to_letter() for d in digits]
        result = []
        for d in digits:
            if isinstance(d, (int, float)):
                if 0 <= d <= 4.25:
                    result.append(cls.from_numeric(d).to_letter())
                else:
                    result.append(d)  # Keep out-of-range values as-is
            else:
                result.append(cls(d).to_letter())  # Process string grades normally
        return result

    @classmethod
    def to_numeric_values(cls, grades):
        if not grades:
            raise ValueError("Grade list cannot be empty")
        return [cls(g).numeric_value for g in grades]

    @classmethod
    def from_numeric(cls, value):
        if not 0 <= value <= 4.25:
            raise ValueError(f"Numeric value {value} out of valid range (0 to 4.25)")
        return cls(str(value))

    def __eq__(self, other):
        '''
        if isinstance(other, LetterGrade):
            return abs(self.numeric_value - other.numeric_value) < 1e-10
        return False
        '''
    
        if isinstance(other, (LetterGrade, str)):
            return self.to_letter() == (other.to_letter() if isinstance(other, LetterGrade) else other)
        elif isinstance(other, (int, float)):
            return abs(self.numeric_value - other) < 1e-10
        return NotImplemented

    def __str__(self):
        return self.to_letter()

    def __repr__(self):
        return f"LetterGrade('{self.original_grade}')"

# Example usage
if __name__ == "__main__":
    # Test composite grade
    composite = LetterGrade('A-/B+')
    print(f"{composite} = {float(composite)}")  #  expected 3.5
    assert LetterGrade('A-/B+').to_letter() == 'A-/B+'
    assert float( LetterGrade('A-/B+') ) == 3.5
    
    # Test numeric to composite (should be B+/B, not B/B+)
    print(f"3.125 to letter: {LetterGrade('3.125')}")  #  expected 'B+/B'
    assert LetterGrade( '3.125' ).to_letter( ) == 'B+/B'    

    # Test addition exceeding A+
    a = LetterGrade('A')
    b_plus = LetterGrade('B+')
    sum_result = (a + b_plus) / 2
    print(f"('A' + 'B+') / 2 = {LetterGrade.from_numeric(sum_result)}")  #  expected 'A/B+'
    assert LetterGrade.from_numeric((a + b_plus)/2).to_letter( ) == 'A/B+'

    # Test subtraction
    diff = a - b_plus
    print( a , type( a ) , b_plus , type( b_plus ) , diff , type( diff ) , LetterGrade( diff ) ,
         LetterGrade( diff ).to_letter( ) )
    print(f"'A' - 'B+' = {diff}")  #  expected 0.75
    assert LetterGrade(a - b_plus).to_letter( ) == 'D-'
    print(f"'A-' + 0.1875 = {LetterGrade('A-')+0.1875}")  #  expected A
    assert ( LetterGrade('A-') + 0.1875 ).to_letter( ) == 'A'    

    # Test addition within range
    a_plus_result = a + 0.25
    print(f"'A' + 0.25 = {a_plus_result}")  #  expected A+
    assert a_plus_result.to_letter( ) == 'A+'    

    # Test five-number summary with in-range digits
    digits_in_range = [4.25, 3.25, 3.75, 2.0, 3.0]
    print(f"Five-number summary of {digits_in_range} (q13='ii', default) = {LetterGrade.five_number_summary(digits_in_range)}")
    
    # Test five-number summary with out-of-range digits
    digits_out_of_range = [212, 10, 50, 70, 164, 96, 144]
    print(f"Five-number summary of {digits_out_of_range} (q13='ii', default) = {LetterGrade.five_number_summary(digits_out_of_range)}")
    
    # Test dot product
    weights = [0.3, 0.5, 0.2]
    grades = ['A', 'B+', 'C']
    dot_result = LetterGrade.dot_product(weights, grades)
    print(f"Dot product of {weights} and {grades} = {dot_result}")

    # Test median with the problematic case
    median_grades = ['A-', 'B+', 'A-', 'A-/B+', 'A-', 'A-', 'A-/B+', 'B+', 'A-/B+', 'A-/A']    
    print(f"Median of {median_grades} = {LetterGrade.median(median_grades)}")  # Expected: A/B+
    assert LetterGrade.median( median_grades ) == 'A/B+'
    
    # Test with corrected typo 'A/A-' instead of 'A-/A'
    median_grades_fixed = ['A-', 'B+', 'A-', 'A-/B+', 'A-', 'A-', 'A-/B+', 'B+', 'A-/B+', 'A/A-']    
    print(f"Median of {median_grades_fixed} = {LetterGrade.median(median_grades_fixed)}")  # Expected: A/B+    
    assert LetterGrade.median(median_grades_fixed) == 'A/B+'
    
    # Test dot product between [0.4, 0.2, 0.2, 0.1, 0.1] ['B', 'B', 'B', 'B', 'C']    
    w , g = [0.4, 0.2, 0.2, 0.1, 0.1] , ['B', 'B', 'B', 'B', 'C']   
    dot_result1 = LetterGrade.dot_product( w , g )
    print( f"Dot product of {w} and {g} = {dot_result1}" )  #  Expected: B
    assert LetterGrade.dot_product( w , g ).to_letter( ) == 'B'
    
    # Test dot product between [0.4, 0.2, 0.2, 0.1, 0.1] ['B+', 'A-/B+', 'C', 'B', 'B']
    w , g = [0.4, 0.2, 0.2, 0.1, 0.1] , ['B+', 'A-/B+', 'C', 'B', 'B']    
    dot_result2 = LetterGrade.dot_product( w , g )
    print( f"Dot product of {w} and {g} = {dot_result2}" )  #  Expected: B
    assert LetterGrade.dot_product( w , g ).to_letter( ) == 'B'
    
    # Test dot product between [0.6, 0.3, 0.1] ['B', 'B', 'C']
    w , g = [0.6, 0.3, 0.1] , ['B', 'B', 'C']    
    dot_result3 = LetterGrade.dot_product( w , g )
    print( f"Dot product of {w} and {g} = {dot_result3}" )  #  Expected: B
    assert LetterGrade.dot_product( w , g ).to_letter( ) == 'B'
    
    # Test dot product between [0.6, 0.3, 0.1] ['B+/B', 'B', 'C']
    w , g = [0.6, 0.3, 0.1] , ['B+/B', 'B', 'C']    
    dot_result6 = LetterGrade.dot_product( w , g )
    print( f"Dot product of {w} and {g} = {dot_result6}" )  #  Expected: B
    assert LetterGrade.dot_product( w , g ).to_letter( ) == 'B'
    
    # Test dot product between [0.6, 0.2, 0.2] ['B', 'B', 'B']
    w , g = [0.6, 0.2, 0.2] , ['B', 'B', 'B']    
    dot_result7 = LetterGrade.dot_product( w , g )
    print( f"Dot product of {w} and {g} = {dot_result7}" )  #  Expected: B
    assert LetterGrade.dot_product( w , g ).to_letter( ) == 'B'
    
    # Test dot product between [0.6, 0.3, 0.1] ['A-/B+', 'A-/B+', 'A+']
    w , g = [0.6, 0.3, 0.1] , ['A-/B+', 'A-/B+', 'A+']    
    dot_result4 = LetterGrade.dot_product( w , g )
    print( f"Dot product of {w} and {g} = {dot_result4}\tShould be A-/B+" )  #  Expected: A-/B+
    ### assert LetterGrade.dot_product( w , g ).to_letter( ) == 'A-/B+'  ###  uncomment after fix
    
    #  Test dot product between [0.4, 0.4, 0.2] ['A-/B+', 'A-/B+', 'A']
    w , g = [0.4, 0.4, 0.2] , ['A-/B+', 'A-/B+', 'A']
    dot_result5 = LetterGrade.dot_product( w , g )
    print( f"Dot product of {w} and {g} = {dot_result5}\tShould be A-/B+" )  #  Expected: A-/B+
    ### assert LetterGrade.dot_product( w , g ).to_letter( ) == 'A-/B+'  ###  uncomment after fix

A-/B+ = 3.5
3.125 to letter: B+/B
('A' + 'B+') / 2 = A/B+
A <class '__main__.LetterGrade'> B+ <class '__main__.LetterGrade'> 0.75 <class 'float'> D- D-
'A' - 'B+' = 0.75
'A-' + 0.1875 = A
'A' + 0.25 = A+
Five-number summary of [4.25, 3.25, 3.75, 2.0, 3.0] (q13='ii', default) = ('C', 'B-/C+', 'B+', 'A', 'A+')
Five-number summary of [212, 10, 50, 70, 164, 96, 144] (q13='ii', default) = (10.0, 50.0, 96.0, 164.0, 212.0)
Dot product of [0.3, 0.5, 0.2] and ['A', 'B+', 'C'] = B+
Median of ['A-', 'B+', 'A-', 'A-/B+', 'A-', 'A-', 'A-/B+', 'B+', 'A-/B+', 'A-/A'] = A/B+
Median of ['A-', 'B+', 'A-', 'A-/B+', 'A-', 'A-', 'A-/B+', 'B+', 'A-/B+', 'A/A-'] = A/B+
Dot product of [0.4, 0.2, 0.2, 0.1, 0.1] and ['B', 'B', 'B', 'B', 'C'] = B
Dot product of [0.4, 0.2, 0.2, 0.1, 0.1] and ['B+', 'A-/B+', 'C', 'B', 'B'] = B
Dot product of [0.6, 0.3, 0.1] and ['B', 'B', 'C'] = B
Dot product of [0.6, 0.3, 0.1] and ['B+/B', 'B', 'C'] = B
Dot product of [0.6, 0.2, 0.2] and ['B', 'B', 'B'] = B
Dot product of [0.6, 0

In [8]:
# test cases:     # Direct expression   #  f: Downloads/ltrGrdCdRq.txt
if ( 1 ) :
    result = (LetterGrade('A') + LetterGrade('B')) / 2
    print(f"Direct ('A' + 'B') / 2 = {result}")  #  Expected:  'A-/B+'
    assert LetterGrade( (LetterGrade('A') + LetterGrade('B')) / 2 ) == LetterGrade( 'A-/B+' )

    # Separate statements
    a = LetterGrade('A')
    b = LetterGrade('B')
    sum_ab = a + b
    print(f"a + b = {sum_ab}, type: {type(sum_ab)}")  #  Expected:  7.0
    assert LetterGrade( a + b ) == 7.0
    div_ab = (a + b) / 2
    print(f"(a + b) / 2 = {div_ab}, type: {type(div_ab)}")  #  Expected:  'A-/B+' LetterGrade
    assert LetterGrade( ( a + b ) / 2 ) == LetterGrade( 'A-/B+' )
    cast_div_ab = LetterGrade.from_numeric(div_ab)  
    print(f"Cast (a + b) / 2 = {cast_div_ab}")  #  Expected:  'A-/B+'
    assert LetterGrade.from_numeric( ( a + b ) / 2 ) == LetterGrade( 'A-/B+' )
    assert LetterGrade.from_numeric( ( a + b ) / 2 ).to_letter( ) == LetterGrade( 'A-/B+' )

    # Multiplication and division
    grade = LetterGrade('B+')  # 3.25

    # Multiplication
    print(grade * 2)      # Returns 6.5 (outside valid range)
    print(grade * 0.8 , LetterGrade( grade * 0.8 ).to_letter( ) == 'B-/C+' , '\tShould be B-/C+' )    # Returns LetterGrade('B-') (2.6 -> B-/C+)
    assert LetterGrade( grade * 2 ) == 6.5
    ### assert LetterGrade( grade * 0.8 ).to_letter( ) == 'B-/C+'  ###  uncomment after fix

    # Division
    print(grade / 2)      # Returns LetterGrade('C/D+') (1.625 -> C/D+)
    print(2 / grade)      # Returns 0.615... (2/3.25) or D-
    assert LetterGrade( grade / 2 ) == LetterGrade( 'C/D+' )
    assert LetterGrade( 2 / grade ) == LetterGrade( 'D-' )
    
    print(f"'A' * 2 = {LetterGrade('A') * 2}")  #  Expected:  8.0
    print(f"'A' / 2 = {LetterGrade('A') / 2}")  #  Expected:  'C'
    assert LetterGrade( 'A' ) * 2 == 8.0
    assert LetterGrade( 'A' ) / 2 == LetterGrade( 'C' )

    print(f"3.125 to letter: {LetterGrade('3.125')}")  #  should be 'B+/B'
    assert LetterGrade('3.125') == 'B+/B'
    print(f"('A' + 'B+') / 2 = {(LetterGrade('A') + LetterGrade('B+')) / 2}")  #  Expected:  'A/B+'
    assert (LetterGrade('A') + LetterGrade('B+')) / 2 == LetterGrade( 'A/B+' )
    
    # Test Problem 1: LetterGrade from LetterGrade object
    lg1 = LetterGrade('A+')
    lg2 = LetterGrade(lg1)
    print(f"LetterGrade from LetterGrade: {lg2} = {float(lg2)}")  #  Expected:  'A+' = 4.25
    assert LetterGrade( LetterGrade( 'A+' ) ) == float( LetterGrade( 'A+' ) )

    # Test Problem 2: LetterGrade('3.4375') equally close to A-/B or A-/B+
    lg3 = LetterGrade('3.4375')
    print(f"LetterGrade('3.4375') = {lg3} (numeric: {float(lg3)})")  #  pick higher tie: Expected:  'A-/B+'
    print(f"3.4375 to letter: {LetterGrade('3.4375')}\tShould be A-/B+")  #  should be 'A-/B+'
    ### assert LetterGrade('3.4375').to_letter( ) == 'A-/B+'  ###  uncomment after fix
    lg5 = LetterGrade('3.5675')
    print(f"LetterGrade('3.5675') = {lg5} (numeric: {float(lg5)})\tShould be A-/B+")  #  pick higher tie: Expected:  'A-/B+'
    print(f"3.4375 to letter: {LetterGrade('3.4375')}")  #  should be 'A-'
    ### assert LetterGrade('3.5675').to_letter( ) == 'A-/B+'  ###  uncomment after fix

    # Additional tests
    print(f"LetterGrade('A') = {LetterGrade('A')}")
    print(f"LetterGrade('A-/B+') = {LetterGrade('A-/B+')}")
    median_result = LetterGrade.median(['A+', 'A-', 'A'])  #  Expected:  'A'
    print(f"Median of ['A+', 'A-', 'A'] = {median_result}, type: {type(median_result)}")  #  Expected: 'A'
    assert LetterGrade.median(['A+', 'A-', 'A']) == 'A'
    
    # Test composite grade
    composite = LetterGrade('A-/B+')
    print(f"{composite} = {float(composite)}")  # Expected: A-/B+ = 3.5
    assert LetterGrade('A-/B+') == 3.5

    # Test 3.125 (should be B+/B)  #  Test numeric to composite (should be B+/B, not B/B+)
    print(f"3.125 to letter: {LetterGrade('3.125')}")  # Expected: B+/B
    assert LetterGrade('3.125') == 'B+/B'

    # Test ('A' + 'B+') / 2 = 3.625 (should be A/B+)  #  Test addition exceeding A+
    a = LetterGrade('A')
    b_plus = LetterGrade('B+')
    sum_result = (a + b_plus) / 2  #  now LetterGrade object
    print(f"('A' + 'B+') / 2 = {LetterGrade.from_numeric(sum_result)}")  # Expected: A/B+
    assert (LetterGrade('A') + LetterGrade('B+'))/2 == 3.625

    # Additional tests
    print(f"LetterGrade('A+') = {LetterGrade('A+')}")
    print(f"LetterGrade('A-/B+') = {LetterGrade('A-/B+')}")
    median_result = LetterGrade.median(['B+', 'C-', 'A'])
    print(f"Median of ['B+', 'C-', 'A'] = {median_result}")  #  Expected:  'B+'
    assert LetterGrade.median(['B+', 'C-', 'A']) == 'B+'
    
    # Test subtraction
    diff = a - b_plus
    print(f"'A' - 'B+' = {diff}")  # Expected:  'D-'
    assert LetterGrade( 'A' ) - LetterGrade( 'B+' ) == LetterGrade( 'D-' )
    print(f"'A' + 0.1875 = {LetterGrade('A')+0.1875}")  #  Expected:  'A+'
    assert LetterGrade( 'A' ) + 0.1875 == LetterGrade( 'A+' )
    print(f"'A-' + 0.1875 = {LetterGrade('A-')+0.1875}")  #  Expected:  'A'
    assert LetterGrade( 'A-' ) + 0.1875 == 'A'

    # Test addition out of range
    print(f"'A+' + 0.25 = {LetterGrade('A+')+0.25}")  #  Expected:  4.5
    assert LetterGrade('A+') + 0.25 == 4.5
    
    # Test five-number summary with in-range digits for (min, Q1, median, Q3, max)
    digits_in_range = [4.25, 3.25, 3.75, 2.0, 3.0]
    print(f"Five-number summary of {digits_in_range} (q13='ii', default) = {LetterGrade.five_number_summary(digits_in_range)}")  # Expected: ('C', 'B', 'B+', 'A-', 'A+')
    print(f"Five-number summary of {digits_in_range} (q13='ie') = {LetterGrade.five_number_summary(digits_in_range, q13='ie')}")  # Expected: ('C', 'B', 'B+', 'A', 'A+')
    print(f"Five-number summary of {digits_in_range} (q13='ee') = {LetterGrade.five_number_summary(digits_in_range, q13='ee')}")  # Expected: ('C', 'B-/C+', 'B+', 'A', 'A+')

    # Test five-number summary with out-of-range digits
    digits_out_of_range = [212, 10, 50, 70, 164, 96, 144]
    print(f"Five-number summary of {digits_out_of_range} (q13='ee', default) = {LetterGrade.five_number_summary(digits_out_of_range)}")  # Expected: (10, 50, 96, 164, 212)
    print(f"Five-number summary of {digits_out_of_range} (q13='ie') = {LetterGrade.five_number_summary(digits_out_of_range, q13='ie')}")  # Expected: (10, 60, 96, 164, 212)
    print(f"Five-number summary of {digits_out_of_range} (q13='ii', default) = {LetterGrade.five_number_summary(digits_out_of_range, q13='ii')}")  # Expected: (10, 60, 96, 154, 212)

    # Test conversion methods
    print(f"Digits to letters: {LetterGrade.to_letter_grades(digits_in_range)}")  # Expected: ['A+', 'B+', 'A-', 'C', 'B']
    grades1 = ['A+', 'B+', 'A-', 'C', 'B']
    print(f"Grades to digits: {LetterGrade.to_numeric_values(grades1)}")  # Expected: [4.25, 3.25, 3.75, 2.0, 3.0]
    
    # Test dot product
    weights = [0.3, 0.5, 0.2]
    grades = ['A', 'B+', 'C']
    dot_result = LetterGrade.dot_product(weights, grades)
    print(f"Dot product of {weights} and {grades} = {dot_result}")  # Expected: 'B+'
    assert LetterGrade.dot_product(weights, grades) == 'B+'

    # Test dot product with out-of-range result
    weights_out = [2.0, 3.0, 1.0]
    grades_out = ['A', 'B+', 'C']
    dot_out_result = LetterGrade.dot_product(weights_out, grades_out)  #  19.75
    print(f"Dot product of {weights_out} and {grades_out} = {dot_out_result}")  # Expected: 19.75 (float)
    LetterGrade.dot_product(weights_out, grades_out) == 19.75
    
    # Test median with a difficult case
    composite_grades = ['A-/B+', 'B+/B ', 'A-/B+', 'A-/B+', 'A-/B+', 'A-/B+', 'B-/C+', 'A-/B+', 'A-/B+', 'A-/B+']
    print(f"Median of {composite_grades} = {LetterGrade.median(composite_grades)}")  # Expected: 'A-/B+'
    assert LetterGrade.median(composite_grades) == 'A-/B+'
    
    # Test additional composite grades
    test_composites = ['A/A-', 'A+/A', 'B-/C+']
    print(f"Median of {test_composites} = {LetterGrade.median(test_composites)}")  # Expected: 'A/A-'
    assert LetterGrade.median(test_composites) == 'A/A-'
    
    # Test median with a problematic case that contains 'A-/A' which should be handled correctly as 'A/A-'
    median_grades = ['A-', 'B+', 'A-', 'A-/B+', 'A-', 'A-', 'A-/B+', 'B+', 'A-/B+', 'A-/A']
    print(f"Median of {median_grades} = {LetterGrade.median(median_grades)}")  # Expected: 'A/B+'
    assert LetterGrade.median(median_grades) == 'A/B+'
    median_result2 = LetterGrade.median(median_grades)
    print(f"Median of {median_grades} = {median_result2}, type: {type(median_result2)}")
    assert LetterGrade.median(median_grades) == 'A/B+'
    
    # Test with corrected typo 'A/A-' instead of 'A-/A'
    median_grades_fixed = ['A-', 'B+', 'A-', 'A-/B+', 'A-', 'A-', 'A-/B+', 'B+', 'A-/B+', 'A/A-']
    print(f"Median of {median_grades_fixed} = {LetterGrade.median(median_grades_fixed)}")  # Expected: 'A/B+'
    
    # Arithmetic operations
    print("=== Arithmetic ===")
    print(f"Direct ('A' + 'B') / 2 = {(LetterGrade('A') + LetterGrade('B')) / 2}")
    a = LetterGrade('A')
    b = LetterGrade('B')
    sum_ab = a + b
    print(f"a + b = {sum_ab}, type: {type(sum_ab)}")
    div_ab = (a + b) / 2
    print(f"(a + b) / 2 = {div_ab}, type: {type(div_ab)}")
    cast_div_ab = LetterGrade.from_numeric(div_ab)
    print(f"Cast (a + b) / 2 = {cast_div_ab}")
    print(f"'A' * 2 = {LetterGrade('A') * 2}")
    print(f"'A' / 2 = {LetterGrade('A') / 2}")
    print(f"('A' + 'B+') / 2 = {(LetterGrade('A') + LetterGrade('B+')) / 2}")

    # Numeric conversions
    print("\n=== Numeric Conversions ===")
    print(f"LetterGrade('B+/B+') = {LetterGrade('B+')}")  #  Expected 'B+' for 'B+/B+' or 'B' for 'B/B'

    # Five-number summary
    print("\n=== Five-Number Summary ===")
    print(f"Five-number summary of ['A+', 'B+', 'A', 'C'] (q13='ii') = {LetterGrade.five_number_summary(['A+', 'B+', 'A', 'C'])}")
    out_of_range = [212, 10, 50, 70, 164]
    print(f"Five-number summary of {out_of_range} = {LetterGrade.five_number_summary(out_of_range)}")

    # List conversions
    print("\n=== List Conversions ===")
    digits = [4.25, 3.25, 2.0]
    print(f"To letter grades: {digits} -> {LetterGrade.to_letter_grades(digits)}")
    grades_list = ['A+', 'B-', 'C+']
    print(f"To numeric values: {grades_list} -> {LetterGrade.to_numeric_values(grades_list)}")
    print("\n=== List No Conversions ===")
    digits = [7.25, 5.25, -2.0]
    print(f"To letter grades: {digits} -> {LetterGrade.to_letter_grades(digits)}")  #  Expected :  [7.25, 5.25, -2.0]
    digits = [7.25, 5.25, -2.0, 3.75, 'A', 'B+']
    print(f"To letter grades: {digits} -> {LetterGrade.to_letter_grades(digits)}")
    # Output: [7.25, 5.25, -2.0, 'A-', 'A', 'B+']

2025-06-15 17:42:54,381 [43633] WARNING  py.warnings:109: [JupyterRequire] /var/folders/bw/f4ppx1nn4wv65wlthyhvkx440000gp/T/ipykernel_43633/1430047503.py:37: RuntimeWarning: Numeric value 7.0 is outside valid range (0–4.25)
  warnings.warn(

2025-06-15 17:42:54,383 [43633] WARNING  py.warnings:109: [JupyterRequire] /var/folders/bw/f4ppx1nn4wv65wlthyhvkx440000gp/T/ipykernel_43633/1430047503.py:37: RuntimeWarning: Numeric value 6.5 is outside valid range (0–4.25)
  warnings.warn(



Direct ('A' + 'B') / 2 = 3.5
a + b = 7.0, type: <class 'float'>
(a + b) / 2 = 3.5, type: <class 'float'>
Cast (a + b) / 2 = A-/B+
6.5
B- False 	Should be B-/C+
C/D+
D-
'A' * 2 = 8.0
'A' / 2 = C
3.125 to letter: B+/B
('A' + 'B+') / 2 = 3.625
LetterGrade from LetterGrade: A+ = 4.25
LetterGrade('3.4375') = B+ (numeric: 3.4375)
3.4375 to letter: B+	Should be A-/B+
LetterGrade('3.5675') = A- (numeric: 3.5675)	Should be A-/B+
3.4375 to letter: B+
LetterGrade('A') = A
LetterGrade('A-/B+') = A-/B+
Median of ['A+', 'A-', 'A'] = A, type: <class 'str'>
A-/B+ = 3.5
3.125 to letter: B+/B
('A' + 'B+') / 2 = A/B+
LetterGrade('A+') = A+
LetterGrade('A-/B+') = A-/B+
Median of ['B+', 'C-', 'A'] = B+
'A' - 'B+' = 0.75
'A' + 0.1875 = A+
'A-' + 0.1875 = A
'A+' + 0.25 = 4.5
Five-number summary of [4.25, 3.25, 3.75, 2.0, 3.0] (q13='ii', default) = ('C', 'B-/C+', 'B+', 'A', 'A+')
Five-number summary of [4.25, 3.25, 3.75, 2.0, 3.0] (q13='ie') = ('C', 'B', 'B+', 'A', 'A+')
Five-number summary of [4.25, 3.25, 3.